In [1]:
import pandas as pd
from shapely.geometry import Point, MultiPolygon
import vaex
from datetime import datetime
import numpy as np
import pyarrow as pa

# Data preparation

In [2]:
df_taxi_trips_all = vaex.open('./data/trips.hdf5')

### Replace spaces and uppercases in column names
column_names = df_taxi_trips_all.column_names
column_names_refactored = [ln.replace(' ', '_').lower() for ln in column_names]

for i, column in enumerate(column_names):
    df_taxi_trips_all.rename(column, column_names_refactored[i])

# correct typo
df_taxi_trips_all.rename("dropoff_centroid__location","dropoff_centroid_location")

# cast timestamp columns to datetime
date_format = "%m/%d/%Y %I:%M:%S %p"
def column_to_datetime(datetime_str):
    return np.datetime64(datetime.strptime(datetime_str, date_format))

df_taxi_trips_all['trip_start_timestamp'] = df_taxi_trips_all['trip_start_timestamp'].apply(column_to_datetime)
df_taxi_trips_all['trip_end_timestamp'] = df_taxi_trips_all['trip_end_timestamp'].apply(column_to_datetime)

In [3]:
# open external data: census tracts
df_census_tracts = vaex.open('./data/chicago_census_tracts.csv')
df_census_tracts.rename("the_geom", "geometry")

# community areas
df_community_areas = vaex.open('./data/community_areas.csv')

In [4]:
total_trips = len(df_taxi_trips_all)
print(f"Total amount of trips: {total_trips:,}")

Total amount of trips: 24,988,003


In [5]:
# Number of trips with trip_miles = 0
zero_trip_miles = len(df_taxi_trips_all[df_taxi_trips_all["trip_miles"] == 0])
print(f"Number of trips with trip_miles = 0: {zero_trip_miles:,}")

# Number of trips with trip_miles = 0 and no difference in pickup and dropoff location
zero_trip_miles_same_loc = len(df_taxi_trips_all[(df_taxi_trips_all["trip_miles"] == 0) & (df_taxi_trips_all["pickup_centroid_location"] == df_taxi_trips_all["dropoff_centroid_location"])])
print(f"Number of trips without trip miles and same location: {zero_trip_miles_same_loc:,}")

Number of trips with trip_miles = 0: 3,003,697
Number of trips without trip miles and same location: 1,387,241


### Removing Trips with Zero Trip Miles and Same Pickup/Dropoff Locations

In [6]:
# drop rows without trip miles and same location
df_non_zero_trip_miles = df_taxi_trips_all[(df_taxi_trips_all["trip_miles"] != 0) | (df_taxi_trips_all["pickup_centroid_location"] != df_taxi_trips_all["dropoff_centroid_location"])]
print(f"Total Trips with Non-Zero Trip Miles and Different Pickup/Dropoff Locations: {len(df_non_zero_trip_miles):,}")

Total Trips with Non-Zero Trip Miles and Different Pickup/Dropoff Locations: 23,600,762


### Insert Missing Census Tract IDs

In [7]:
# Calculate the percentage of missing pickup_census_tract IDs
percentage_missing_pickup_census_tract = (len(df_non_zero_trip_miles[df_non_zero_trip_miles["pickup_census_tract"].isnan()]) / len(df_non_zero_trip_miles)) * 100
percentage_missing_dropoff_census_tract = (len(df_non_zero_trip_miles[df_non_zero_trip_miles["dropoff_census_tract"].isnan()]) / len(df_non_zero_trip_miles)) * 100

# Round the percentage to two decimal places
percentage_rounded_pickup = round(percentage_missing_pickup_census_tract, 2)
percentage_rounded_dropoff = round(percentage_missing_dropoff_census_tract, 2)

# Display the result
print(f"Percentage of missing pickup_census_tract IDs: {percentage_rounded_pickup}%")
print(f"Percentage of missing pickup_census_tract IDs: {percentage_rounded_dropoff}%")

Percentage of missing pickup_census_tract IDs: 30.76%
Percentage of missing pickup_census_tract IDs: 31.13%


In [8]:
# Drop all rows where both "X_census_tract" and "X_centroid_location" are null
# We keep rows WITH "X_centroid_location" and WITHOUT "pickup_census_tract" to craft census tracts
df_cleaned_census_and_location = df_non_zero_trip_miles.dropna(column_names=["pickup_census_tract", "pickup_centroid_location"], how="all")
df_cleaned_census_and_location = df_cleaned_census_and_location.dropna(column_names=["dropoff_census_tract", "dropoff_centroid_location"], how="all")
print(f"Total Trips without Rows with Null Values in \"X_centroid_location\" AND \"X_census_tract\": {len(df_cleaned_census_and_location):,}")

Total Trips without Rows with Null Values in "X_centroid_location" AND "X_census_tract": 21,170,643


In [16]:
df_no_census_tract_both = df_cleaned_census_and_location[df_cleaned_census_and_location["pickup_census_tract"].isnan() | df_cleaned_census_and_location["dropoff_census_tract"].isnan()]
print(f"Number of Rows where \"X_census_tract\" is Null: {len(df_no_census_tract_both):,}")

Number of Rows where "X_census_tract" is Null: 5,000,517


In [20]:
# Assuming you have one dataframe containing null values for both pickup_census_tract and dropoff_census_tract
df_no_census_tract_both = df_no_census_tract_both.to_pandas_df()

# Convert the pickup_centroid_location in the dataframe to Point geometries
df_no_census_tract_both['pickup_centroid_location'] = df_no_census_tract_both.apply(
    lambda row: Point(row['pickup_centroid_longitude'], row['pickup_centroid_latitude']), axis=1
)

# Prepare a function to find the census tract for a given point
def find_census_tract(point, census_tract_df):
    for index, row in census_tract_df.iterrows():
        if point.within(row['geometry']):
            return row['GEOID10']
    return None

# Create dictionaries to store the census tract IDs for pickup and dropoff points
pickup_census_tract_ids = {}
dropoff_census_tract_ids = {}

# Iterate through each row of the dataframe and find the corresponding census tract IDs for both pickup and dropoff
for index, row in df_no_census_tract_both.iterrows():
    pickup_location = row['pickup_centroid_location']
    dropoff_location = row['pickup_centroid_location']

    if pickup_location not in pickup_census_tract_ids:
        pickup_census_tract_ids[pickup_location] = find_census_tract(pickup_location, df_census_tracts)

    if dropoff_location not in dropoff_census_tract_ids:
        dropoff_census_tract_ids[dropoff_location] = find_census_tract(dropoff_location, df_census_tracts)

# Update the "pickup_census_tract" column
df_no_census_tract_both['pickup_census_tract'] = df_no_census_tract_both['pickup_centroid_location'].map(pickup_census_tract_ids)

# Update the "dropoff_census_tract" column
df_no_census_tract_both['dropoff_census_tract'] = df_no_census_tract_both['pickup_centroid_location'].map(dropoff_census_tract_ids)

KeyboardInterrupt: 

In [15]:
df_census_not_nan = df_cleaned_census_and_location.dropna(column_names=["pickup_census_tract", "dropoff_census_tract"])
df_census_not_nan = vaex.from_pandas(df_census_not_nan)

16170126

In [ ]:
df_inserted_census_tracts = df_census_not_nan.concat(df_no_census_tract_both)

In [ ]:
# check which values contain NA and NaN values
column_names = df_inserted_census_tracts.get_column_names()

for column in column_names:
    if df_inserted_census_tracts[column].isna().sum() > 0:
        print(f"Column '{column}' contains NA or NaN values with a number of " + str(df_inserted_census_tracts[column].isna().sum()) + " rows.")

In [ ]:
# We decide to drop all columns with NA or NaN values for consistent analysis across different tasks
print("Total number of rows before dropping NA and NaN: " + str(df_inserted_census_tracts.count()))

df_no_missing_values = df_inserted_census_tracts.dropnan()
print("Number of rows without NaN-values: " + str(df_no_missing_values.count()))

df_no_missing_values = df_no_missing_values.dropna()
print("Number of rows without NA-values: " + str(df_no_missing_values.count()))

### Check for consistency

In [9]:
# check if trip ids are unique
print("Trip IDs are unique?: " + str(len(df_no_missing_values) == df_no_missing_values['trip_id'].nunique()))


In [10]:
# check for consistency in community areas
print("Number of community areas: " + str(df_community_areas.count()))
print("Number of community areas in taxi trip data without NaN-values: " + str(df_no_missing_values.pickup_community_area.nunique(dropnan=True)))

In [11]:
# check for consistency in community areas
community_areas = df_community_areas.AREA_NUMBE.values.unique()
community_areas_int = set([area.as_py() for area in community_areas])

community_areas_pickup = df_no_missing_values.pickup_community_area.unique(dropnan=True)
community_areas_pickup_int = set([int(area) for area in community_areas_pickup])

community_areas_dropoff = df_no_missing_values.pickup_community_area.unique(dropnan=True)
community_areas_dropoff_int = set([int(area) for area in community_areas_dropoff])

print("Do the pickup community area IDs in the taxi trip data match the community area dataset? ",community_areas_pickup_int.issubset(community_areas_int))
print("Do the dropoff community area IDs in the taxi trip data match the community area dataset? ",community_areas_dropoff_int.issubset(community_areas_int))

In [12]:
# check for consistency in census tracts
print("Number of census tracts: " + str(df_census_tracts.count()))
print("Number of pickup census tracts in filtered taxi trip data: " + str(df_no_missing_values.pickup_census_tract.nunique()))
print("Number of dropoff census tracts in filtered taxi trip data: " + str(df_no_missing_values.dropoff_census_tract.nunique()))

In [13]:
# check if the census tracts in the taxi data match the census tracts dataset
df_census_tracts.GEOID10.values
census_tracts = set([id.as_py() for id in df_census_tracts.GEOID10.values])
census_tracts_taxi_pickups = set([int(id) for id in df_no_missing_values.pickup_census_tract.unique(dropnan=True)])
census_tracts_taxi_dropoffs = set([int(id) for id in df_no_missing_values.dropoff_census_tract.unique(dropnan=True)])

print("Do the pickup census tract IDs in the taxi trip data match the census tract dataset? ",census_tracts_taxi_pickups.issubset(census_tracts))
print("Do the dropoff census tract IDs in the taxi trip data match the census tract dataset? ",census_tracts_taxi_dropoffs.issubset(census_tracts))

In [14]:
# create columns for hourly discretization
df_no_missing_values["trip_start_hour"] = df_no_missing_values.trip_start_timestamp.dt.hour
df_no_missing_values["trip_end_hour"] = df_no_missing_values.trip_end_timestamp.dt.hour

In [15]:
# export prepared dataframe
df_no_missing_values.export_hdf5('./data/trips_prepared.hdf5', progress=True)

export(hdf5) [#############---------------------------] 33.33% estimated time:     0.44s =  0.0m =  0.0h 

export(hdf5) [########################################] 100.00% elapsed time  :  1904.98s =  31.7m =  0.5h                                                 
 